# discrete linear quadratic gaussian control - LQG

- here we presents robust discrete optimal control LQG
- discrete version of LQG can be represented by following block diagram 

- LQG solves problem, ** when no full state x is observed**
- typical aplication is, we can measure position of servo, but no velocitny and current
- this observation is called y
- to reconstruct full state x_hat is used Kalman observer

- reconstructed state x_hat is input into doscrete LQR


![image](../../doc/diagrams/control-lqg_discrete.png)

## three spring connected wheels

- 6th order dynamics
- 2 inputs
- state : wheels position and velocity (pos_0, pos_1, pos_2, v0, v1, v2)
- observation : only wheels position (pos_0, pos_1, pos_2)

In [1]:
import numpy
import scipy
import matplotlib.pyplot as plt

from wheels import *

dt = 0.001 

# create dynamical system
ds = Wheels(dt)
# print system dynamics

print("continuous model")
print(ds.mat_a.shape)
print(ds.mat_b.shape)
print(ds.mat_c.shape)
print()

# discretise model using bilinear transform
# x(n+1) = A x(n) + B u(n)
i = numpy.eye(ds.mat_a.shape[0])
 
tmp_a = numpy.linalg.inv(i - (0.5*dt)*ds.mat_a)
tmp_b = i + (0.5*dt)*ds.mat_a

mat_a_disc    = tmp_a@tmp_b
mat_b_disc    = (tmp_a*dt)@ds.mat_b
mat_c_disc    = ds.mat_c

print("discrete model")
print(mat_a_disc)
print(mat_b_disc) 
print(mat_c_disc) 
print()



continuous model
(6, 6)
(6, 2)
(3, 6)

discrete model
[[ 9.99988097e-01  5.91014797e-06  2.00252025e-11  9.99843406e-04
   2.95454808e-09  1.00106391e-14]
 [ 5.90998671e-06  9.99987313e-01  6.77650560e-06  2.93309671e-09
   9.99815690e-04  3.38758883e-09]
 [ 7.53878946e-08  6.73896017e-06  9.99993224e-01 -6.33107921e-06
   3.36888043e-09  9.99800651e-04]
 [-2.38051256e-02  1.18202959e-02  4.00504049e-08  9.99686811e-01
   5.90909616e-06  2.00212783e-11]
 [ 1.18199734e-02 -2.53730198e-02  1.35530112e-02  5.86619342e-06
   9.99631379e-01  6.77517767e-06]
 [ 1.50775789e-04  1.34779203e-02 -1.35528076e-02 -1.26621584e-02
   6.73776085e-06  9.99601302e-01]]
[[ 5.99239481e-06  6.34007146e-17]
 [ 1.75790263e-11  2.14547293e-11]
 [-3.79442681e-08  6.33207079e-06]
 [ 1.19847896e-02  1.26801429e-13]
 [ 3.51580525e-08  4.29094586e-08]
 [-7.58885362e-05  1.26641416e-02]]
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]



## controller synthetis


In [2]:
'''
solve the discrete time lqr controller for
x(n+1) = A x(n) + B u(n)
y(n) = C x(n)
cost = sum x[n].T*Q*x[n] + u[n].T*R*u[n]
'''
def solve_discrete_lqr(a, b, c, q, r):

    n = a.shape[0]  #system order
    m = b.shape[1]  #inputs count
    k = c.shape[0]  #outputs count

    #matrix augmentation with integral action
    a_aug = numpy.zeros((n+k, n+k))
    b_aug = numpy.zeros((n+k, m))
    q_aug = numpy.zeros((n+k, n+k))

    
    a_aug[0:n, 0:n] = a 

    #add integrator into augmented a matrix
    for i in range(k):
        a_aug[i + n, i]     = 1.0
        a_aug[i + n, i + n] = 1.0

    b_aug[0:n,0:m]  = b

    #project Q matrix to output, and fill augmented q matrix
    tmp = (c@q).sum(axis=1)
    for i in range(k):
        q_aug[n+i][n+i] = tmp[i]

    
    # discrete-time algebraic Riccati equation solution
    p = scipy.linalg.solve_discrete_are(a_aug, b_aug, q_aug, r)

    # compute the LQR gain
    ki_tmp =  numpy.linalg.inv(r)@(b_aug.T@p)

    #truncated small elements
    ki_tmp[numpy.abs(ki_tmp) < 10**-10] = 0

    #split ki for k and integral action part ki
    k   = ki_tmp[:, 0:a.shape[0]]
    ki  = ki_tmp[:, a.shape[0]:]

    return k, ki

'''
compute kalman gain matrix for observer : 
x_hat(n+1) = Ax_hat(n) + Bu(n) + K(y(n) - Cx_hat(n))
'''
def solve_kalman_gain(a, c, q, r):
    p = scipy.linalg.solve_discrete_are(a.T, c.T, q, r) 
    k = p@c.T@scipy.linalg.inv(c@p@c.T + r)

    return k

#create loss weighting matrices (diagonal)
q = numpy.diag([1.0, 1.0, 1.0, 0.0, 0.0, 0.0])
r = numpy.diag([1.0, 1.0])

#find LQR controll matrices
k, ki = solve_discrete_lqr(mat_a_disc, mat_b_disc, mat_c_disc, q, r)

print("k  = ", k)
print("ki = ", ki)


q_noise = 0.001*numpy.eye(mat_a_disc.shape[0]) 
r_noise = 0.1*numpy.eye(mat_c_disc.shape[0]) 

f = solve_kalman_gain(mat_a_disc, mat_c_disc, q_noise, r_noise)
print("f = ", f)
    

k  =  [[ 90.90181447  97.24612395 -10.48649973   3.95197126  13.50655118
   -0.37518399]
 [ 21.57272368 162.95014683  92.35598956  -0.38861534  22.12977377
    3.7723544 ]]
ki =  [[ 0.94395438  0.30431481 -0.25353547]
 [ 0.06990641  0.5064285   0.8871317 ]]
f =  [[ 9.52410919e-02  8.65005612e-05 -7.53199249e-04]
 [ 8.65005612e-05  9.55740697e-02  1.35547533e-04]
 [-7.53199249e-04  1.35547533e-04  9.88771439e-02]
 [ 1.25543699e-02  9.13591360e-03 -8.00883239e-02]
 [ 8.98894083e-03  4.73478005e-02  1.52838570e-02]
 [-8.16561381e-02  1.38015538e-02  4.02998208e-01]]


## running demo


In [3]:

#run demo

#random initial state
x = numpy.random.randn(ds.mat_a.shape[0], 1)

#required output, 1 rad for all wheels
yr = numpy.array([[1.0, 1.0, 1.0]]).T

#observed state
x_hat = numpy.zeros((ds.mat_a.shape[0], 1))

#initial error integral
integral_action = numpy.zeros((ds.mat_b.shape[1], 1))

u = numpy.zeros((ds.mat_b.shape[1], 1))


#initial error integral
integral_action = numpy.zeros((ds.mat_b.shape[1], 1))


#plant output
y = ds.y

disturbance = 0

n = 0

while True:
    # kalman observer
    # only y is known, and using knowledge of dynamics, 
    # the full state x_hat can be reconstructed
    prediction_error = y - ds.mat_c@x_hat
    x_hat = mat_a_disc@x_hat + mat_b_disc@u + f@prediction_error


    #integral action
    error = yr - y
    integral_action+= ki@error

    #LQR controll law
    u = -k@x_hat + integral_action

    x, y = ds.step(x, u)
    
    if n%10 == 0:
        ds.render()

    #change required state
    if n%2000 == 0:
        yr*= -1
        
    n+= 1
  